Yahoo Finance agent

In [6]:
import os
import sys
from dotenv import load_dotenv

## Add root directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

load_dotenv()
assert os.environ["LANGCHAIN_API_KEY"], "Please set the LANGCHAIN_API_KEY environment variable"
assert os.environ["OPENAI_API_KEY"], "Please set the OPENAI_API_KEY environment variable"
assert os.environ["TAVILY_API_KEY"], "Please set the TAVILY_API_KEY environment variable"

from langchain_openai import ChatOpenAI
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent

from pprint import pprint

openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])
DATA_DIR = "../../../data/processed"

In [4]:
llm = ChatOpenAI(temperature=0.0)
tools = [YahooFinanceNewsTool()]
graph = create_react_agent(llm, tools)

In [5]:
messages = [SystemMessage(content="""
    You are an AI assistant to a team of traders and analysts. Help them find any relevant news for specified stocks.
    Pay attention to the specified date ranges. If none are specified, retrieve the latest information you have.
    If you cannot find all the required information, it is ok, just respond with what you have.
""")]
prompt = "AAPL, 10 October 2024"
messages.append(HumanMessage(content=prompt))

response = graph.invoke({"messages": [("user", "Are you able to retrieve news within a specified date range?")]})
pprint(response["messages"][-1].content)

('I can retrieve news articles for a specific company, but I may not be able '
 'to filter them by a specific date range. Would you like me to retrieve the '
 'latest news for a particular company?')


Tavily Search agent

In [ ]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(
    api_wrapper=search,
    search_depth="advanced",
    verbose=True,
    include_answer=True,
    # include_domains=["finance.yahoo.com", "bloomberg.com", "reuters.com"]    
)

graph = create_react_agent(llm, [tavily_tool])

In [35]:
messages = [SystemMessage(content="""
    You are an AI assistant to a team of traders and analysts. Help them find relevant news or information to support their needs
    or address their problems. Pay attention to the specified date ranges. Run a search for each day of the date range, 
    and compile the results. If no dates are specified, retrieve the latest information you have.
    If you cannot find all the required information, it is ok, just respond with what you have.
    Make sure you answer the user's question directly in your response.
""")]
prompt = "TSMC stocks have been on the rise, what are some reasons why?"
messages.append(HumanMessage(content=prompt))

response = graph.invoke({"messages": messages})
print(response["messages"][-1].content)

content='[{"url": "https://finance.yahoo.com/news/2-straightforward-reasons-taiwan-semiconductor-145300871.html", "content": "Jan 3, 2024 · Thanks to a boost from AI-related hype and anticipated demand, TSMC\'s stock price increased over 40% in 2023. Even so, here are three reasons"}, {"url": "https://finance.yahoo.com/news/prediction-taiwan-semiconductor-manufacturing-soar-161000115.html", "content": "Aug 18, 2024 · It\'s been a great five years for TSMC stock with its 330% gain, and the next five years look just as promising. The reason is artificial intelligence (AI) and"}, {"url": "https://finance.yahoo.com/news/1-favorable-trend-explains-99-150800839.html", "content": "Sep 20, 2024 · Shares of Taiwan Semiconductor Manufacturing (NYSE: TSM) have soared over the last year, largely due to the soaring demand for cutting-edge chips."}, {"url": "https://finance.yahoo.com/news/why-taiwan-semiconductor-stock-gaining-172300360.html", "content": "Mar 8, 2024 · Shares of Taiwan Semiconductor

Pandas dataframe agent